# Link region data from the Office for National Statistics

Make a single file that has all of the information on LSOA, LHB, SICBL, ICB, and ISDN.

Make a second smaller file that drops the LSOA and contains only unique regions (SICBL and LHB).

For Wales, the region is Local Health Board (LHB). For England, we are using Integrated Care Boards (ICB) and their smaller units Sub Integrated Care Board Loations (SICBL).

## Data sources

Open Geography Portal:

+ Wales `Output_Areas_(2011)_to_Local_Health_Boards_(December_2020)_Lookup_in_Wales.csv`
+ England & Wales `Output_Area_to_LSOA_to_MSOA_to_Local_Authority_District_(December_2017)_Lookup_with_Area_Classifications_in_Great_Britain.csv`
+ England `LSOA11_LOC22_ICB22_LAD22_EN_LU.csv`

Custom files:

We don't have access to any official list of which SICBL belong to which Integrated Stroke Delivery Network (ISDN), but we have made an unofficial list.

Data source: [Building Integrated Stroke Delivery Network (ISDN) footprint geometry](https://github.com/samuel-book/geography_data/blob/mt_catchment/geojson/isdn_boundaries.ipynb)

Note, 22nd Feb 2024: this link will expire when I merge branches in the repository. The notebook `isdn_boundaries.ipynb` exists somewhere in https://github.com/samuel-book/geography_data/.

## Notebook setup

In [1]:
import pandas as pd
import geopandas
import os

In [2]:
dir_geojson = '../../data_geojson'
dir_data = 'data_input'
dir_ons = 'ons_data'

file_oa_to_lhb = 'Output_Areas_(2011)_to_Local_Health_Boards_(December_2020)_Lookup_in_Wales.csv'
file_oa_to_lsoa = 'Output_Area_to_LSOA_to_MSOA_to_Local_Authority_District_(December_2017)_Lookup_with_Area_Classifications_in_Great_Britain.csv'
file_lsoa_to_icb = 'LSOA11_LOC22_ICB22_LAD22_EN_LU.csv'
file_sicbl_to_isdn = 'SICBL_ICB_ISDN.csv'

file_lsoa_output = 'regions_lsoa_ew.csv'
file_output = 'regions_ew.csv'

## Link Welsh LSOA to LHB

We have a list linking OA to LSOA and a list linking OA to LHB, so use the OA column as a step to link LSOA and LHB.

In [3]:
path_to_oa_to_lhb = os.path.join(dir_ons, file_oa_to_lhb)
df_oa_lhb = pd.read_csv(path_to_oa_to_lhb)

In [4]:
df_oa_lhb.head()

,FID,OA11CD,LHB20CD,LHB20NM,LHB20NMW
0,1,W00000052,W11000023,Betsi Cadwaladr University Health Board,Bwrdd Iechyd Prifysgol Betsi Cadwaladr
1,2,W00000104,W11000023,Betsi Cadwaladr University Health Board,Bwrdd Iechyd Prifysgol Betsi Cadwaladr
2,3,W00000053,W11000023,Betsi Cadwaladr University Health Board,Bwrdd Iechyd Prifysgol Betsi Cadwaladr
3,4,W00000054,W11000023,Betsi Cadwaladr University Health Board,Bwrdd Iechyd Prifysgol Betsi Cadwaladr
4,5,W00000105,W11000023,Betsi Cadwaladr University Health Board,Bwrdd Iechyd Prifysgol Betsi Cadwaladr


In [5]:
path_to_oa_to_lsoa = os.path.join(dir_ons, file_oa_to_lsoa)
df_oa_lsoa = pd.read_csv(path_to_oa_to_lsoa)

In [6]:
df_oa_lsoa.head()

,OA11CD,OAC11CD,OAC11NM,LSOA11CD,LSOA11NM,SOAC11CD,SOAC11NM,MSOA11CD,MSOA11NM,LAD17CD,LAD17NM,LACCD,LACNM,RGN11CD,RGN11NM,CTRY11CD,CTRY11NM,FID
0,E00060343,7d1,Ageing Communities and Families,E01011966,Hartlepool 006B,5b,Aspiring urban households,E02002488,Hartlepool 006,E06000001,Hartlepool,6a2r,Mining Legacy,E12000001,North East,E92000001,England,1
1,E00174083,7d1,Ageing Communities and Families,E01011974,Hartlepool 005B,4b,Constrained renters,E02002487,Hartlepool 005,E06000001,Hartlepool,6a2r,Mining Legacy,E12000001,North East,E92000001,England,2
2,E00060349,6a4,Ageing in Suburbia,E01011965,Hartlepool 006A,8b,Ageing suburbanites,E02002488,Hartlepool 006,E06000001,Hartlepool,6a2r,Mining Legacy,E12000001,North East,E92000001,England,3
3,E00060418,6a4,Ageing in Suburbia,E01011983,Hartlepool 006C,8a,Affluent communities,E02002488,Hartlepool 006,E06000001,Hartlepool,6a2r,Mining Legacy,E12000001,North East,E92000001,England,4
4,E00060255,8c1,Ageing Industrious Workers,E01011950,Hartlepool 008A,4a,Challenged white communities,E02002490,Hartlepool 008,E06000001,Hartlepool,6a2r,Mining Legacy,E12000001,North East,E92000001,England,5


In [7]:
df_lsoa_lhb = pd.merge(
    df_oa_lhb[['OA11CD', 'LHB20CD', 'LHB20NM']],
    df_oa_lsoa[['OA11CD', 'LSOA11CD', 'LSOA11NM']],
    on='OA11CD', how='left'
)

# Drop the unwanted OA column:
df_lsoa_lhb = df_lsoa_lhb.drop('OA11CD', axis='columns')
# Remove duplicates (multiple OA in each LSOA):
df_lsoa_lhb = df_lsoa_lhb.drop_duplicates()

In [8]:
df_lsoa_lhb.head()

,LHB20CD,LHB20NM,LSOA11CD,LSOA11NM
0,W11000023,Betsi Cadwaladr University Health Board,W01000012,Isle of Anglesey 006B
1,W11000023,Betsi Cadwaladr University Health Board,W01000021,Isle of Anglesey 002C
9,W11000023,Betsi Cadwaladr University Health Board,W01000022,Isle of Anglesey 004A
10,W11000023,Betsi Cadwaladr University Health Board,W01000013,Isle of Anglesey 005B
20,W11000023,Betsi Cadwaladr University Health Board,W01000023,Isle of Anglesey 007D


Useful data:
+ LHB20CD
+ LHB20NM

In [9]:
cols_to_keep = ['LSOA11NM', 'LSOA11CD', 'LHB20CD', 'LHB20NM']

df_lsoa_lhb = df_lsoa_lhb[cols_to_keep]

## Link English LSOA to SICBL

In [10]:
path_to_lsoa_to_icb = os.path.join(dir_ons, file_lsoa_to_icb)
df_lsoa_icb = pd.read_csv(path_to_lsoa_to_icb)

In [11]:
df_lsoa_icb.head()

,LSOA11CD,LSOA11NM,LOC22CD,LOC22CDH,LOC22NM,ICB22CD,ICB22CDH,ICB22NM,LAD22CD,LAD22NM
0,E01012367,Halton 007A,E38000068,01F,NHS Cheshire and Merseyside ICB - 01F,E54000008,QYG,NHS Cheshire and Merseyside Integrated Care Board,E06000006,Halton
1,E01012368,Halton 003A,E38000068,01F,NHS Cheshire and Merseyside ICB - 01F,E54000008,QYG,NHS Cheshire and Merseyside Integrated Care Board,E06000006,Halton
2,E01012369,Halton 005A,E38000068,01F,NHS Cheshire and Merseyside ICB - 01F,E54000008,QYG,NHS Cheshire and Merseyside Integrated Care Board,E06000006,Halton
3,E01012370,Halton 007B,E38000068,01F,NHS Cheshire and Merseyside ICB - 01F,E54000008,QYG,NHS Cheshire and Merseyside Integrated Care Board,E06000006,Halton
4,E01012371,Halton 016A,E38000068,01F,NHS Cheshire and Merseyside ICB - 01F,E54000008,QYG,NHS Cheshire and Merseyside Integrated Care Board,E06000006,Halton


n.b. the columns called LOC are for SICBL.

Useful data:
+ LOC22CD / NM
+ ICB22CD / NM

In [12]:
cols_to_keep = ['LSOA11NM', 'LSOA11CD', 'LOC22CD', 'LOC22NM', 'ICB22CD', 'ICB22NM']

df_lsoa_icb = df_lsoa_icb[cols_to_keep]

## Link English LSOA to ISDN

Load in the ISDN data:

In [13]:
path_to_sicbl_to_isdn = os.path.join(dir_data, file_sicbl_to_isdn)

df_sicbl_isdn = pd.read_csv(path_to_sicbl_to_isdn)

In [14]:
df_sicbl_isdn.head()

,LOC22CD,LOC22CDH,LOC22NM,ICB22CD,ICB22CDH,ICB22NM,NHSER22CD,NHSER22CDH,NHSER22NM,ISDN
0,E38000240,93C,NHS North Central London ICB - 93C,E54000028,QMJ,NHS North Central London Integrated Care Board,E40000003,Y56,London,London
1,E38000255,A3A8R,NHS North East London ICB - A3A8R,E54000029,QMF,NHS North East London Integrated Care Board,E40000003,Y56,London,London
2,E38000256,W2U3Z,NHS North West London ICB - W2U3Z,E54000027,QRV,NHS North West London Integrated Care Board,E40000003,Y56,London,London
3,E38000244,72Q,NHS South East London ICB - 72Q,E54000030,QKK,NHS South East London Integrated Care Board,E40000003,Y56,London,London
4,E38000245,36L,NHS South West London ICB - 36L,E54000031,QWE,NHS South West London Integrated Care Board,E40000003,Y56,London,London


In [15]:
df_lsoa_icb = pd.merge(
    df_lsoa_icb, df_sicbl_isdn[['LOC22CD', 'ISDN']],
    left_on='LOC22CD', right_on='LOC22CD', how='left'
)

In [16]:
df_lsoa_icb.head()

,LSOA11NM,LSOA11CD,LOC22CD,LOC22NM,ICB22CD,ICB22NM,ISDN
0,Halton 007A,E01012367,E38000068,NHS Cheshire and Merseyside ICB - 01F,E54000008,NHS Cheshire and Merseyside Integrated Care Board,Cheshire and Merseyside
1,Halton 003A,E01012368,E38000068,NHS Cheshire and Merseyside ICB - 01F,E54000008,NHS Cheshire and Merseyside Integrated Care Board,Cheshire and Merseyside
2,Halton 005A,E01012369,E38000068,NHS Cheshire and Merseyside ICB - 01F,E54000008,NHS Cheshire and Merseyside Integrated Care Board,Cheshire and Merseyside
3,Halton 007B,E01012370,E38000068,NHS Cheshire and Merseyside ICB - 01F,E54000008,NHS Cheshire and Merseyside Integrated Care Board,Cheshire and Merseyside
4,Halton 016A,E01012371,E38000068,NHS Cheshire and Merseyside ICB - 01F,E54000008,NHS Cheshire and Merseyside Integrated Care Board,Cheshire and Merseyside


## Simple merge of data

This simple merge has a couple of problems.

English and Welsh health board areas have no overlap, so the combined DataFrame contains a lot of missing values.

And this would need a bit of further work to sort out the two LSOA11NM columns.

In [17]:
df_combo = pd.merge(df_lsoa_icb, df_lsoa_lhb, on='LSOA11CD', how='left')

In [18]:
df_combo.head().T

,0,1,2,3,4
LSOA11NM_x,Halton 007A,Halton 003A,Halton 005A,Halton 007B,Halton 016A
LSOA11CD,E01012367,E01012368,E01012369,E01012370,E01012371
LOC22CD,E38000068,E38000068,E38000068,E38000068,E38000068
LOC22NM,NHS Cheshire and Merseyside ICB - 01F,NHS Cheshire and Merseyside ICB - 01F,NHS Cheshire and Merseyside ICB - 01F,NHS Cheshire and Merseyside ICB - 01F,NHS Cheshire and Merseyside ICB - 01F
ICB22CD,E54000008,E54000008,E54000008,E54000008,E54000008
ICB22NM,NHS Cheshire and Merseyside Integrated Care Board,NHS Cheshire and Merseyside Integrated Care Board,NHS Cheshire and Merseyside Integrated Care Board,NHS Cheshire and Merseyside Integrated Care Board,NHS Cheshire and Merseyside Integrated Care Board
ISDN,Cheshire and Merseyside,Cheshire and Merseyside,Cheshire and Merseyside,Cheshire and Merseyside,Cheshire and Merseyside
LSOA11NM_y,NaN,NaN,NaN,NaN,NaN
LHB20CD,NaN,NaN,NaN,NaN,NaN
LHB20NM,NaN,NaN,NaN,NaN,NaN


## Rearrange region data

Instead, change the data so that there is a single column for region (combined ICB22NM and LHB20NM), region code (ICB22CD and LHB20CD), and region type.

Set both DataFrames to have the same column names:

In [19]:
col_dict_eng = {
    'LOC22CD': 'region_code',
    'LOC22NM': 'region',
    'LSOA11CD': 'LSOA_code',
    'LSOA11NM': 'LSOA',
    'ICB22NM': 'ICB',
    'ICB22CD': 'ICB_code',
}
col_dict_wal = {
    'LHB20CD': 'region_code',
    'LHB20NM': 'region',
    'LSOA11CD': 'LSOA_code',
    'LSOA11NM': 'LSOA',
}

df_lsoa_icb = df_lsoa_icb.rename(columns=col_dict_eng)
df_lsoa_lhb = df_lsoa_lhb.rename(columns=col_dict_wal)

Add an extra column for region type and country.

Although you can tell the country directly from the region type, this option is more explicit.

At this point also change the inexplicable "LOC" to "SICBL".

In [20]:
df_lsoa_icb['region_type'] = 'SICBL'
df_lsoa_icb['country'] = 'England'

df_lsoa_lhb['region_type'] = 'LHB'
df_lsoa_lhb['country'] = 'Wales'

Stack both DataFrames on top of each other:

In [21]:
# Need the same index column for concat() to work.
df_lsoa_icb = df_lsoa_icb.set_index('LSOA_code')
df_lsoa_lhb = df_lsoa_lhb.set_index('LSOA_code')

df_regions = pd.concat((df_lsoa_icb, df_lsoa_lhb), axis='rows')

df_regions = df_regions.reset_index()

In [22]:
df_regions

,LSOA_code,LSOA,region_code,region,ICB_code,ICB,ISDN,region_type,country
0,E01012367,Halton 007A,E38000068,NHS Cheshire and Merseyside ICB - 01F,E54000008,NHS Cheshire and Merseyside Integrated Care Board,Cheshire and Merseyside,SICBL,England
1,E01012368,Halton 003A,E38000068,NHS Cheshire and Merseyside ICB - 01F,E54000008,NHS Cheshire and Merseyside Integrated Care Board,Cheshire and Merseyside,SICBL,England
2,E01012369,Halton 005A,E38000068,NHS Cheshire and Merseyside ICB - 01F,E54000008,NHS Cheshire and Merseyside Integrated Care Board,Cheshire and Merseyside,SICBL,England
3,E01012370,Halton 007B,E38000068,NHS Cheshire and Merseyside ICB - 01F,E54000008,NHS Cheshire and Merseyside Integrated Care Board,Cheshire and Merseyside,SICBL,England
4,E01012371,Halton 016A,E38000068,NHS Cheshire and Merseyside ICB - 01F,E54000008,NHS Cheshire and Merseyside Integrated Care Board,Cheshire and Merseyside,SICBL,England
...,...,...,...,...,...,...,...,...,...
34748,W01001865,Cardiff 022C,W11000029,Cardiff and Vale University Health Board,NaN,NaN,NaN,LHB,Wales
34749,W01001890,Cardiff 021D,W11000029,Cardiff and Vale University Health Board,NaN,NaN,NaN,LHB,Wales
34750,W01001893,Cardiff 010B,W11000029,Cardiff and Vale University Health Board,NaN,NaN,NaN,LHB,Wales
34751,W01001892,Cardiff 020D,W11000029,Cardiff and Vale University Health Board,NaN,NaN,NaN,LHB,Wales


In [23]:
cols_order = [
    'LSOA', 'LSOA_code', 'region', 'region_code', 'region_type',
    'country', 'ICB', 'ICB_code', 'ISDN'
]
df_regions = df_regions[cols_order]

Rename columns:

In [24]:
cols_dict = dict()
for col in df_regions.columns:
    # Change to lower case:
    new_name = col.casefold()
    # Replace spaces with underscores:
    new_name = new_name.replace(' ', '_')
    # Add to the dictionary:
    cols_dict[col] = new_name

df_regions = df_regions.rename(columns=cols_dict)

In [25]:
df_regions.head()

,lsoa,lsoa_code,region,region_code,region_type,country,icb,icb_code,isdn
0,Halton 007A,E01012367,NHS Cheshire and Merseyside ICB - 01F,E38000068,SICBL,England,NHS Cheshire and Merseyside Integrated Care Board,E54000008,Cheshire and Merseyside
1,Halton 003A,E01012368,NHS Cheshire and Merseyside ICB - 01F,E38000068,SICBL,England,NHS Cheshire and Merseyside Integrated Care Board,E54000008,Cheshire and Merseyside
2,Halton 005A,E01012369,NHS Cheshire and Merseyside ICB - 01F,E38000068,SICBL,England,NHS Cheshire and Merseyside Integrated Care Board,E54000008,Cheshire and Merseyside
3,Halton 007B,E01012370,NHS Cheshire and Merseyside ICB - 01F,E38000068,SICBL,England,NHS Cheshire and Merseyside Integrated Care Board,E54000008,Cheshire and Merseyside
4,Halton 016A,E01012371,NHS Cheshire and Merseyside ICB - 01F,E38000068,SICBL,England,NHS Cheshire and Merseyside Integrated Care Board,E54000008,Cheshire and Merseyside


## Output file 1: LSOA to region

In [26]:
df_regions_lsoa = df_regions[['lsoa', 'lsoa_code', 'region', 'region_code', 'region_type']]

In [27]:
df_regions_lsoa

,lsoa,lsoa_code,region,region_code,region_type
0,Halton 007A,E01012367,NHS Cheshire and Merseyside ICB - 01F,E38000068,SICBL
1,Halton 003A,E01012368,NHS Cheshire and Merseyside ICB - 01F,E38000068,SICBL
2,Halton 005A,E01012369,NHS Cheshire and Merseyside ICB - 01F,E38000068,SICBL
3,Halton 007B,E01012370,NHS Cheshire and Merseyside ICB - 01F,E38000068,SICBL
4,Halton 016A,E01012371,NHS Cheshire and Merseyside ICB - 01F,E38000068,SICBL
...,...,...,...,...,...
34748,Cardiff 022C,W01001865,Cardiff and Vale University Health Board,W11000029,LHB
34749,Cardiff 021D,W01001890,Cardiff and Vale University Health Board,W11000029,LHB
34750,Cardiff 010B,W01001893,Cardiff and Vale University Health Board,W11000029,LHB
34751,Cardiff 020D,W01001892,Cardiff and Vale University Health Board,W11000029,LHB


In [28]:
df_regions_lsoa.to_csv(file_lsoa_output, index=False)

## Output file 2: region to other regions

In [29]:
df_regions_small = df_regions.drop(['lsoa', 'lsoa_code'], axis='columns')
df_regions_small = df_regions_small.drop_duplicates()

In [30]:
df_regions_small

,region,region_code,region_type,country,icb,icb_code,isdn
0,NHS Cheshire and Merseyside ICB - 01F,E38000068,SICBL,England,NHS Cheshire and Merseyside Integrated Care Board,E54000008,Cheshire and Merseyside
79,NHS Cheshire and Merseyside ICB - 02E,E38000194,SICBL,England,NHS Cheshire and Merseyside Integrated Care Board,E54000008,Cheshire and Merseyside
206,NHS Cheshire and Merseyside ICB - 27D,E38000233,SICBL,England,NHS Cheshire and Merseyside Integrated Care Board,E54000008,Cheshire and Merseyside
652,NHS Cheshire and Merseyside ICB - 01J,E38000091,SICBL,England,NHS Cheshire and Merseyside Integrated Care Board,E54000008,Cheshire and Merseyside
750,NHS Cheshire and Merseyside ICB - 99A,E38000101,SICBL,England,NHS Cheshire and Merseyside Integrated Care Board,E54000008,Cheshire and Merseyside
...,...,...,...,...,...,...,...
33293,Hywel Dda University Health Board,W11000025,LHB,Wales,NaN,NaN,NaN
33565,Swansea Bay University Health Board,W11000031,LHB,Wales,NaN,NaN,NaN
33799,Cwm Taf Morgannwg University Health Board,W11000030,LHB,Wales,NaN,NaN,NaN
34046,Aneurin Bevan University Health Board,W11000028,LHB,Wales,NaN,NaN,NaN


In [31]:
df_regions_small.to_csv(file_output, index=False)

This final dataframe contains most of the same information as the starting SICBL to ISDN DataFrame, but it is worth making and saving this copy because the column names match the LSOA version.